In [1]:
from distributions.distribution_by_sequence import DistributionBySequence
from distributions.distribution import Distribution
from distributions.sequence import Sequence
from algorithms.semi_bandit_exp3 import SemiBanditExp3
from algorithms.full_bandit_exp3 import FullBanditExp3
from algorithms.semi_bandit_ftrl import SemiBanditFTRL
from algorithms.full_bandit_exp3_inv import FullBanditExp3Inv
from algorithms.real_lin_exp3 import RealLinExp3
from algorithms.uniform_random import UniformRandom
from algorithms.non_contextual_exp3 import NonContextualExp3
from experiment_manager.experiment_manager import ExperimentManager

from misc.tensor_helpers import *

from distributions.actionsets.msets import MSets

from distributions.contexts.binary_context import BinaryContext
from distributions.thetas.single_hole import SingleHole
from distributions.thetas.independent_bernoulli import IndependentBernoulli

rng = np.random.default_rng()

In [8]:
algo = FullBanditExp3Inv()
algo = RealLinExp3()
#algo = SemiBanditExp3()

length = 10000
d = 3
K = 2
actionset = MSets(K, 1)

epsilon = 0.25 * np.min([np.sqrt(K / length), 1])
p = np.zeros((d, K)) + 0.5
for i in range(d):
    p[i, 0] = 0.3

dist_lower_bound = Distribution(BinaryContext(d), IndependentBernoulli(d, K, p), actionset)

seq = dist_lower_bound.generate(length, rng, rng)
algo.set_constants(rng, seq)
print("sigma:", seq.sigma, "m:", seq.m, "beta:", algo.beta, "gamma:", algo.gamma, "eta", algo.eta)
loss, _, _, _ = algo.run_on_sequence(rng, seq)
loss_of_optimal_policy, _, _ = seq.find_optimal_policy()

comperator = UniformRandom()
comperator.set_constants(rng, seq)
loss_comperator, _, _, _ = comperator.run_on_sequence(rng, seq)
print("regret", loss - loss_of_optimal_policy, loss_comperator - loss_of_optimal_policy)


sigma: 1.0 m: 1 beta: 0.5 gamma: 0.03377508689746394 eta 0.0010063304877787227
regret 914.0 1046.0


In [3]:
algo = RealLinExp3()
algo.set_constants(rng, seq)

seq.reset()
context, _, _, done = seq.get_next(None)

losses = []
probability_array = []
action_array = []
while not done:
    probabilities = algo.get_policy(context)
    probability_array.append(probabilities)

    print("Start next")
    print(context, probabilities)
    action_index = rng.choice(np.arange(seq.actionset.number_of_actions), p=probabilities)
    print(action_index, seq.actionset[action_index])
    action_array.append(action_index)

    print("\n")
    next_context, loss, loss_vec, done = seq.get_next(seq.actionset[action_index])
    print(loss, loss_vec)
    loss_vec[~seq.actionset[action_index]] = 0
    print(loss, loss_vec)
    if algo.full_bandit:
        algo.observe_loss(loss, context, action_index)
    else:
        algo.observe_loss_vec(loss_vec, context, action_index)

    print(algo.theta_estimates)
    print("\n\n")
    losses.append(loss)
    context = next_context

    if seq.current_index == 10:
        1/0


Start next
[1.] [0.5 0.5]
1 [False  True]


0.0 [0. 0.]
0.0 [0. 0.]
[[0.]
 [0.]]



Start next
[1.] [0.5 0.5]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[0.]
 [2.]]



Start next
[1.] [0.50094047 0.49905953]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[0.        ]
 [4.00376897]]



Start next
[1.] [0.50188271 0.49811729]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[0.        ]
 [6.01132826]]



Start next
[1.] [0.50282671 0.49717329]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[0.        ]
 [8.02269938]]



Start next
[1.] [0.50377248 0.49622752]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[ 0.        ]
 [10.03790403]]



Start next
[1.] [0.50472003 0.49527997]
0 [ True False]


1.0 [1. 0.]
1.0 [1. 0.]
[[ 1.98129644]
 [10.03790403]]



Start next
[1.] [0.50378843 0.49621157]
1 [False  True]


1.0 [0. 1.]
1.0 [0. 1.]
[[ 1.98129644]
 [12.05317343]]



Start next
[1.] [0.50473601 0.49526399]
1 [False  True]


0.0 [0. 0.]
0.0 [0. 0.]
[[ 1.98129644]
 [12.05317343]]





ZeroDivisionError: division by zero